# Import packages

In [1]:
# Numerical Operations
import math
import numpy as np

# Reading/Writing Data
import pandas as pd
import os
import csv

# Pytorch
import torch 
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, random_split

# Some Utility Functions

In [2]:
def same_seed(seed): 
    '''Fixes random number generator seeds for reproducibility.'''
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

In [3]:
def random_spatial_sequence(split_rate, full_sequence):
    counts = len(full_sequence)
    len1 = int(counts * split_rate)
    len2 = counts - len1
    index1 = np.random.choice(full_sequence, len1, replace=False)
    index2 = np.setdiff1d(full_sequence, index1)
    print('**************************Data Spliting***************************')
    print('Spliting Rate: ', split_rate)
    print(len1, 'of Dataset1: ',index1)
    print(len2, 'of Dataset2: ',index2)
    print('**************************Data Spliting***************************')
    return index1, index2

In [4]:
# spatial_sequence = [2, 8, 9, 10, 11, 12, 15, 20, 25, 30, 34, 40, 44]
# print(len(spatial_sequence))
# random_spatial_sequence(0.7, spatial_sequence)

In [5]:
def collate_fn(batch):
#     print(batch)
#     print(len(batch[0]))
    # 从每个样本的字典中获取处理结果、标签和其他数据，并将它们存储在同一个字典中
    processed_data = [torch.FloatTensor(sample["processed_data"]) for sample in batch]
    label_data = [sample["label_data"] for sample in batch]
    
    # 将所有需要传递的数据都保存在同一个字典中
    batch_dict = {"processed_data": torch.stack(processed_data),
                  "label_data": label_data}
    
    return batch_dict

In [6]:
def calculate_upscaling_sm(sm_bar, sm_bar_sd, ati, ati_bar, ati_bar_sd):
    return sm_bar + sm_bar_sd * (ati - ati_bar) / ati_bar_sd

In [7]:
def predict(test_loader, model, device):
    model.eval() # Set your model to evaluation mode.
    preds = []
    for x in tqdm(test_loader):
        x = x.to(device)                        
        with torch.no_grad():                   
            pred = model(x)                     
            preds.append(pred.detach().cpu())   
    preds = torch.cat(preds, dim=0).numpy()  
    return preds

# Dataset

In [8]:
class SMAPDataset(Dataset):
    '''
    root: root of input data
    temperal_sequence: the sequence of valid day
    spatial_sequence: the sequence of valid SMAPID
    test: the flag to identify if it is the test dataset
    '''
    def __init__(self, root, temperal_sequence, spatial_sequence):
        # variables for input
        self.smap = []
        self.texture = []
        
        # variables for output
        self.sm = []
        self.smap_unorm = []
        self.ati = [] # contains [ati, atim, atisd] in each element
        
        print('***************************Load data path******************************')
        for i in temperal_sequence: # for example: 2015187
            print('_______________________________' + str(i) + '_______________________________')
            for j in spatial_sequence: # for example: 1
                print('_____________________________smap cell: ' + str(j) + '_____________________________')
                # add path for input variables
                self.smap.append(root + 'INPUT\\SMAP\\' + i + '\\' + str(j) + '.npy')
                self.texture.append(root + 'INPUT\\TEXTURE\\' + str(j) + '.npy')
                # display adding path
                print((root + 'INPUT\\SMAP\\' + i + '\\' + str(j) + '.npy'))
                print((root + 'INPUT\\TEXTURE\\' + str(j) + '.npy'))
                print(os.path.exists(root + 'INPUT\\SMAP\\' + i + '\\' + str(j) + '.npy'))
                print(os.path.exists(root + 'INPUT\\TEXTURE\\' + str(j) + '.npy'))
                      
                # one smap to many in-situ sm
                self.smap_unorm.append(root + 'LABEL\\SMAP\\' + i + '\\' + str(j) + '.npy')
                smap_to_insitu = np.load(root + "LABEL\\SMAPID2INSITUID\\" + str(j) + '.npy')
                insitu_sm_list = []
                insitu_ati_list = []
                for _id in smap_to_insitu:
                    insitu_sm_list.append(root + "LABEL\\SM\\" + i + "\\" + str(_id) + ".npy")
                    insitu_ati_list.append(root + "LABEL\\ATI\\" + i + "\\" + str(_id) + ".npy")
                    # display adding path
                    print((root + "LABEL\\SM\\" + i + "\\" + str(_id) + ".npy"))
                    print((root + "LABEL\\ATI\\" + i + "\\" + str(_id) + ".npy"))
                    print(os.path.exists(root + "LABEL\\SM\\" + i + "\\" + str(_id) + ".npy"))
                    print(os.path.exists(root + "LABEL\\ATI\\" + i + "\\" + str(_id) + ".npy"))
                      
                # add the data of insitu in insitu_list
                self.sm.append(insitu_sm_list)
                self.ati.append(insitu_ati_list)    
                      
    def __getitem__(self, idx):
        smap_path = self.smap[idx]
        smap_unorm_path = self.smap_unorm[idx]
        texture_path = self.texture[idx]
        smap = np.load(smap_path)
        smap_unorm = np.load(smap_unorm_path)
        texture = np.load(texture_path)
        
        data_pkg = {'processed_data': [], 'label_data': []}
        
        # choose flatten as the way to concatenate the input feature
        x = self.__flatten__(smap, texture)
        data_pkg['processed_data'] = x
        
        sm_list = self.sm[idx]
        ati_list = self.ati[idx]
        y = [] # y -> [[sm, smap, ati], ...], sm -> [float], smap -> [float],1 ati -> [ati, atim, atisd]
        for i in range(len(sm_list)):
            sm_path = sm_list[i]
            ati_path = ati_list[i]
            sm = np.load(sm_path, allow_pickle=True)
            ati = np.load(ati_path, allow_pickle=True)
            data_pkg['label_data'].append([sm, smap_unorm, ati])      # other_data -> [[sm, smap, ati], ...], 
                                                                    # sm -> [float]
                                                                    # smap -> [float], 
                                                                    # ati -> [ati, atim, atisd]
            
            # test: each element in list of batch should be of equal size
#             break
        return data_pkg

    def __len__(self):
        return len(self.smap)

    ### the way to concatenate input data
    def __flatten__(self, smap, texture):
        # normalization is done before loading
        texture_flat = texture.flatten()
        return  np.concatenate((smap, texture_flat), axis=0)
    
    def get_input_shape(self, idx):
        data_pkg = self.__getitem__(0)
        return data_pkg['processed_data'].shape

# Neural Network Model
Try out different model architectures by modifying the class below.

In [9]:
class My_Model(nn.Module):
    def __init__(self, input_dim):
        super(My_Model, self).__init__()
        # TODO: modify model's structure, be aware of dimensions. 
        self.layers = nn.Sequential(
            nn.Linear(input_dim, 16),
            nn.ReLU(),
            nn.Linear(16, 8),
            nn.ReLU(),
            nn.Linear(8, 1)
        )

    def forward(self, x):
        x = self.layers(x)
        x = x.squeeze(1) # (B, 1) -> (B)
        return x

# Self-defined Loss Function


In [10]:
def my_Loss(pred, label_data):
    criterion = nn.MSELoss(reduction='mean')
    loss = 0
    for i, y in enumerate(pred):      # each calculation in a batch
        # calculate the high resolution sm for each in situ
        for situ_pkg in label_data[i]: # each calculation in a smap
            sm_situ = situ_pkg[0][0]
            sm_bar = situ_pkg[1][0]
            ati =  situ_pkg[2][0]
            atim = situ_pkg[2][1]
            atisd = situ_pkg[2][2]
            sm_pred = calculate_upscaling_sm(sm_bar, y, ati, atim, atisd)
#             _loss = criterion(sm_pred, torch.FloatTensor([sm_situ]))
            _loss = criterion(sm_pred, torch.tensor(sm_situ, dtype=torch.float32))
            loss = (loss + _loss)
    loss = loss / config['batch_size']
    return loss

# Training Loop

In [11]:
def trainer(train_loader, valid_loader, model, config, device):

    criterion = my_Loss # Define your loss function, do not modify this.
    optimizer = torch.optim.SGD(model.parameters(), lr=config['learning_rate'], momentum=0.9) 

    if not os.path.isdir(config['root'] + '\\OUTPUT\\MODELS\\'):
        os.mkdir(config['root'] + '\\OUTPUT\\MODELS\\') # Create directory of saving models.

    n_epochs, best_loss, step, early_stop_count = config['n_epochs'], math.inf, 0, 0

    for epoch in range(n_epochs):
        model.train() # Set your model to train mode.
        loss_record = []

        for data_pkg in train_loader:
            x = data_pkg['processed_data']
            optimizer.zero_grad()               # Set gradient to zero.
#             x, y = x.to(device), y.to(device)   # Move your data to device. 
            pred = model(x)
            loss = criterion(pred, data_pkg['label_data'])
            loss.backward()                     # Compute gradient(backpropagation).
            optimizer.step()                    # Update parameters.
            step += 1
            loss_record.append(loss.detach().item())
            
        mean_train_loss = sum(loss_record)/len(loss_record)
#         print('Mean Loss for Epoch {}: {}'.format(epoch, mean_train_loss))

        model.eval() # Set your model to evaluation mode.
        loss_record = []
        for data_pkg in valid_loader:
            x = data_pkg['processed_data']
#             x, y = x.to(device), y.to(device)   # Move your data to device. 
            with torch.no_grad():
                pred = model(x)
                loss = criterion(pred, data_pkg['label_data'])
            loss_record.append(loss.item())
            
        mean_valid_loss = sum(loss_record)/len(loss_record)
        print(f'Epoch [{epoch+1}/{n_epochs}]: Train loss: {mean_train_loss:.4f}, Valid loss: {mean_valid_loss:.4f}')

        if mean_valid_loss < best_loss:
            best_loss = mean_valid_loss
            torch.save(model.state_dict(), config['root'] + '\\OUTPUT\\MODELS\\' + str(best_loss) + '.ckpt') # Save your best model
            print('Saving model with loss {:.3f}...'.format(best_loss))
            early_stop_count = 0
        else: 
            early_stop_count += 1

        if early_stop_count >= config['early_stop']:
            print('\nModel is not improving, so we halt the training session.')
            return

# Configurations
`config` contains hyper-parameters for training and the path to save your model.

In [12]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
config = {
    'seed': 52013141,      # Your seed number, you can pick your lucky number. :)
    'test_ratio': 0.2,
    'valid_ratio': 0.2,   # validation_size = train_size * valid_ratio
    'n_epochs': 300,     # Number of epochs.            
    'batch_size': 5, 
    'learning_rate': 1e-5,              
    'early_stop': 10,    # If model has not improved for this many consecutive epochs, stop training.     
    'root': 'D:\\1GRADUATED\\paper\\downscaling_data\\Soil_moisture_downscale_czt\\DATASET\\'
}

# Dataloader
Read data from files and set up training, validation, and testing sets. You do not need to modify this part.

In [13]:
# Set seed for reproducibility
same_seed(config['seed'])

In [14]:
# Get the id sequence of all valid SMAP 不同日期的SMAPID2INSITUID不同
directory = config['root'] + 'LABEL\\SMAPID2INSITUID\\2015187'
full_spatial_sequence_smap = sorted([int(f.split('.')[0]) for f in os.listdir(directory) if f.endswith('.npy')]) # !!!! read out of order
print(len(full_spatial_sequence_smap), 'of Full Spatial Sequence: ', full_spatial_sequence_smap)

16 of Full Spatial Sequence:  [9, 10, 15, 16, 17, 18, 20, 26, 27, 29, 33, 40, 43, 44, 45, 51]


In [19]:
# Split the train\valid\test dataset by spatial dimension
train_spatial_seq, test_spatial_seq = random_spatial_sequence(1-config['test_ratio'], full_spatial_sequence_smap)
train_spatial_seq, valid_spatial_seq = random_spatial_sequence(1-config['valid_ratio'], train_spatial_seq)

# Initialize the dataset
temperal_seq = ['20151015']
train_dataset = SMAPDataset(config['root'], temperal_seq, train_spatial_seq)
valid_dataset = SMAPDataset(config['root'], temperal_seq, valid_spatial_seq)
test_dataset = SMAPDataset(config['root'], temperal_seq, test_spatial_seq)

# Pytorch data loader loads pytorch dataset into batches.
train_loader = DataLoader(train_dataset, batch_size=config['batch_size'], shuffle=True, pin_memory=True, collate_fn=collate_fn)
valid_loader = DataLoader(valid_dataset, batch_size=config['batch_size'], shuffle=True, pin_memory=True, collate_fn=collate_fn)
test_loader = DataLoader(test_dataset, batch_size=config['batch_size'], shuffle=False, pin_memory=True, collate_fn=collate_fn)

**************************Data Spliting***************************
Spliting Rate:  0.8
12 of Dataset1:  [10 26 44 29 43 16  9 33 45 40 51 17]
4 of Dataset2:  [15 18 20 27]
**************************Data Spliting***************************
**************************Data Spliting***************************
Spliting Rate:  0.8
9 of Dataset1:  [17 43 45 40 16 51 29 26 33]
3 of Dataset2:  [ 9 10 44]
**************************Data Spliting***************************
***************************Load data path******************************
_______________________________20151015_______________________________
_____________________________smap cell: 17_____________________________
D:\1GRADUATED\paper\downscaling_data\Soil_moisture_downscale_czt\DATASET\INPUT\SMAP\20151015\17.npy
D:\1GRADUATED\paper\downscaling_data\Soil_moisture_downscale_czt\DATASET\INPUT\TEXTURE\17.npy
True
True


FileNotFoundError: [Errno 2] No such file or directory: 'D:\\1GRADUATED\\paper\\downscaling_data\\Soil_moisture_downscale_czt\\DATASET\\LABEL\\SMAPID2INSITUID\\17.npy'

# Data visualization

In [ ]:
import matplotlib.pyplot as plt

In [ ]:

# 训练集sm站点数据
sm = [x['label_data'][0][0] for x in train_dataset]
x = [x for x in range(len(sm))]
y = sm

# 绘制散点图
plt.scatter(x, y, c='red', s=10, alpha=1)

# 添加标签和标题
plt.ylim(0, 1)
plt.xlabel('X')
plt.ylabel('Y')
plt.title('SM In-Situ')

# 显示图像
plt.show()

# 训练集SMAP数据
smap = [x['label_data'][0][1] for x in train_dataset]
x = [x for x in range(len(sm))]
y = smap

# 绘制散点图
plt.scatter(x, y, c='red', s=10, alpha=1)

# 添加标签和标题
plt.ylim(0, 1)
plt.xlabel('X')
plt.ylabel('Y')
plt.title('SMAP')

# 显示图像
plt.show()


# 创建数据
ati = [x['label_data'][0][2] for x in train_dataset]
print(ati)
x = [x for x in range(len(ati))]

y1 = [x[0] for x in ati]
y2 = [x[1] for x in ati]
y3 = [x[2] for x in ati]

# 创建子图
fig, (ax1, ax2, ax3) = plt.subplots(nrows=1, ncols=3, figsize=(10, 5))

# 在第一个子图中绘制ATI
ax1.scatter(x, y1)
ax1.set_title('ATI')
ax1.set_ylim([-0.02,0.05])

# 在第二个子图中绘制ATIM
ax2.scatter(x, y2)
ax2.set_title('ATIM')
ax2.set_ylim([-0.02,0.05])

# 在第三个子图中绘制ATISD
ax3.scatter(x, y3)
ax3.set_title('ATISD')
ax3.set_ylim([-0.02,0.05])

# 显示图像
plt.show()

In [ ]:
# 验证集sm站点数据
sm = [x['label_data'][0][0] for x in valid_dataset]
x = [x for x in range(len(sm))]
y = sm

# 绘制散点图
plt.scatter(x, y, c='red', s=10, alpha=1)

# 添加标签和标题
plt.ylim(0, 1)
plt.xlabel('X')
plt.ylabel('Y')
plt.title('SM In-Situ')

# 显示图像
plt.show()

# 验证集SMAP数据
smap = [x['label_data'][0][1] for x in valid_dataset]
x = [x for x in range(len(sm))]
y = smap

# 绘制散点图
plt.scatter(x, y, c='red', s=10, alpha=1)

# 添加标签和标题
plt.ylim(0, 1)
plt.xlabel('X')
plt.ylabel('Y')
plt.title('SMAP')

# 显示图像
plt.show()


# 创建数据
ati = [x['label_data'][0][2] for x in valid_dataset]
print(ati)
x = [x for x in range(len(ati))]

y1 = [x[0] for x in ati]
y2 = [x[1] for x in ati]
y3 = [x[2] for x in ati]

# 创建子图
fig, (ax1, ax2, ax3) = plt.subplots(nrows=1, ncols=3, figsize=(10, 5))

# 在第一个子图中绘制sin函数
ax1.scatter(x, y1)
ax1.set_title('ATI')
ax1.set_ylim([-0.02,0.05])

# 在第二个子图中绘制cos函数
ax2.scatter(x, y2)
ax2.set_title('ATIM')
ax2.set_ylim([-0.02,0.05])

# 在第三个子图中绘制tan函数
ax3.scatter(x, y3)
ax3.set_title('ATISD')
ax3.set_ylim([-0.02,0.05])

# 显示图像
plt.show()

# Start training!

In [ ]:
model = My_Model(input_dim=train_dataset.get_input_shape(0)[0]).to(device) # put your model and data on the same computation device.
print(train_dataset.get_input_shape(0)[0])
print(model)
trainer(train_loader, valid_loader, model, config, device)

# Testing
The predictions of your model on testing set will be stored at `pred.csv`.

In [ ]:
def save_pred(preds, file):
    ''' Save predictions to specified file '''
    with open(file, 'w') as fp:
        writer = csv.writer(fp)
        writer.writerow(['id', 'tested_positive'])
        for i, p in enumerate(preds):
            writer.writerow([i, p])

model = My_Model(input_dim=x_train.shape[1]).to(device)
model.load_state_dict(torch.load(config['save_path']))
preds = predict(test_loader, model, device) 
save_pred(preds, 'pred.csv')         

# Reference
This notebook uses code written by Heng-Jui Chang @ NTUEE (https://github.com/ga642381/ML2021-Spring/blob/main/HW01/HW01.ipynb)